In [1]:
%set_env DYLD_LIBRARY_PATH=/Users/hosborn/python/MultiNest/lib

env: DYLD_LIBRARY_PATH=/Users/hosborn/python/MultiNest/lib


In [1]:
import json
import numpy
from numpy import log, exp, pi
import scipy.stats, scipy
import pymultinest
import matplotlib.pyplot as plt

# we define the problem: we need a prior function which maps from [0:1] to the parameter space

# we only have one parameter, the position of the gaussian (ndim == 1)
# map it from the unity interval 0:1 to our problem space 0:2 under a uniform prior
def prior(cube, ndim, nparams):
    cube[0] = cube[0] * 2

# our likelihood function consists of 6 gaussians modes (solutions) at the positions
positions = numpy.array([0.1, 0.2, 0.5, 0.55, 0.9, 1.1])
width = 0.01

def loglike(cube, ndim, nparams):
    # get the current parameter (is between 0:2 now)
    pos = cube[0]
    likelihood = exp(-0.5 * ((pos - positions) / width)**2) / (2*pi*width**2)**0.5
    return log(likelihood.mean())

# number of dimensions our problem has
parameters = ["position"]
n_params = len(parameters)

# run MultiNest
pymultinest.run(loglike, prior, n_params, verbose = True)

AttributeError: dlsym(RTLD_DEFAULT, run): symbol not found

In [ ]:
x = numpy.linspace(0, 1, 400)
ydata = None # loaded below

noise = 0.1

np.loadtxt("/Users/hosborn/python/PyMultiNest/pymultinest/")

# model for 2 gaussians, same width, fixed offset
def model(pos1, width, height1, height2):
    pos2 = pos1 + 0.05
    return  height1 * scipy.stats.norm.pdf(x, pos1, width) + \
        height2 * scipy.stats.norm.pdf(x, pos2, width)

# a more elaborate prior
# parameters are pos1, width, height1, [height2]
def prior(cube, ndim, nparams):
    #cube[0] = cube[0]            # uniform prior between 0:1
    cube[1] = 10**(cube[1]*8 - 4) # log-uniform prior between 10^-4 and 10^4
    cube[2] = 10**(cube[2]*4 - 4) # log-uniform prior between 10^-4 and 1
    if ndim < 4:
        return
    cube[3] = 10**(cube[3]*4 - 4) # log-uniform prior between 10^-4 and 1


def loglike(cube, ndim, nparams):
    pos1, width, height1 = cube[0], cube[1], cube[2]
    height2 = cube[3] if ndim > 3 else 0
    ymodel = model(pos1, width, height1, height2)
    loglikelihood = (-0.5 * ((ymodel - ydata) / noise)**2).sum()
    return loglikelihood

parameters = ["pos1", "width", "height1"]
n_params = len(parameters)

pymultinest.run(loglike, prior, n_params, outputfiles_basename='test_1_', resume = False, verbose = True)


In [ ]:
from scipy.stats import norm,beta,truncnorm

def transform_uniform(x,a,b):
    return a + (b-a)*x

def transform_loguniform(x,a,b):
    la=np.log(a)
    lb=np.log(b)
    return np.exp(la + x*(lb-la))

def transform_normal(x,mu,sigma):
    return norm.ppf(x,loc=mu,scale=sigma)

def transform_beta(x,a,b):
    return beta.ppf(x,a,b)

def transform_truncated_normal(x,mu,sigma,a=0.,b=1.):
    ar, br = (a - mu) / sigma, (b - mu) / sigma
    return truncnorm.ppf(x,ar,br,loc=mu,scale=sigma)


In [ ]:
def multinest():
    parameters = ["x", "y"]
    n_params = len(parameters)
    args=(x_data, y_data, err_data)
    
    per_index=-8/3
    
    def myprior(cube, ndim, nparams):
        # do prior transformation
        start=None

        ######################################
        #   Intialising Stellar Params:
        ######################################
        #Using log rho because otherwise the distribution is not normal:
        cube_indeces={}
        cube_indeces['logrho_S']=nparams;nparams+=1
        cube[0] = transform_normal(cube[0],mu=np.log(self.rhostar[0]), 
                                    sd=np.average(abs(self.rhostar[1:]/self.rhostar[0])))
        cube_indeces['Rs']=nparams;nparams+=1
        cube[cube_indeces['Rs']] = transform_normal(cube[cube_indeces['Rs']],mu=self.Rstar[0],
                                                    sd=np.average(abs(self.Rstar[1:])))
        # The baseline flux
        cube_indeces['mean']=nparams;nparams+=1
        cube_indeces['mean']=transform_normal(cube[cube_indeces['mean']],mu=np.median(self.lc['flux'][self.lc['mask']]),
                                              sd=np.std(self.lc['flux'][self.lc['mask']]))

        # The 2nd light (not third light as companion light is not modelled) 
        # This quantity is in delta-mag
        if useL2:
            cube_indeces['deltamag_contam']=nparams;nparams+=1
            cube[cube_indeces['deltamag_contam']] = transform_uniform(cube[cube_indeces['deltamag_contam']], lower=-20.0, upper=20.0)
            mult = (1+np.power(2.511,-1*cube[cube_indeces['deltamag_contam']])) #Factor to multiply normalised lightcurve by
        else:
            mult=1.0

        print("Forming Pymc3 model with: monos:",self.monos,"multis:",self.multis,"duos:",self.duos)

        ######################################
        #     Initialising Periods & tcens
        ######################################
        for pl in self.multis+self.monos+self.duos:
            tcen=self.planets[pl]['tcen']
            tdur=self.planets[pl]['tdur']
            cube_indeces['t0_'+pl]=nparams;nparams+=1
            cube[cube_indeces['t0_'+pl]]=transform_truncated_normal(cube[cube_indeces['t0_'+pl]],
                                                                    mu=tcen,sigma=tdur*0.1,
                                                                    a=tcen-tdur*0.5,b=tcen+tdur*0.5)
            if pl in self.monos:
                cube_indeces['per_'+pl]=nparams;nparams+=1
                #Need to loop through possible period gaps, according to their prior probability *density* (hence the final term diiding by gap width)
                rel_gap_prob = (self.planets[pl]['per_gaps'][:,0]**(-5/3)-self.planets[pl]['per_gaps'][:,1]**(-5/3))/self.planets[pl]['per_gaps'][:,2]
                rel_gap_prob /= np.sum(rel_gap_prob)
                rel_gap_prob = np.hstack((0.0,np.cumsum(rel_gap_prob)))
                incube=cube[cube_indeces['per_'+pl]]
                outcube=incube[:]
                #looping through each gpa, cutting the "cube" up according to their prior probability and making a p~P^-8/3 distribution for each gap:
                for i in range(len(rel_gap_prob)-1):
                    ind_min=np.power(self.planets[pl]['per_gaps'][i,1]/self.planets[pl]['per_gaps'][i,0],per_index)
                    outcube[(incube>relgap_prob[i])&(incube<=rel_gap_prob[i+1])]=np.power(((1-ind_min)*(incube[(incube>rel_gap_prob[i])&(incube<=rel_gap_prob[i+1])]-rel_gap_prob[i])/(rel_gap_prob[i+1]-rel_gap_prob[i])+ind_min),1/per_index)*self.planets[pl]['per_gaps'][i,0]
                cube[cube_indeces['per_'+pl]]=outcube
            # The period distributions of monotransits are tricky as we often have gaps to contend with
            # We cannot sample the full period distribution while some regions have p=0.
            # Therefore, we need to find each possible period region and marginalise over each

            if pl in self.duos:
                #In the case of a duotransit, we have a discrete series of possible periods between two know transits.
                #If we want to model this duo transit exactly like the first (as a bounded normal)
                # we can work out the necessary periods to cause these dips
                cube_indeces['t0_2_'+pl]=nparams;nparams+=1
                tcen2=self.planets[pls]['tcen_2']
                cube[cube_indeces['t0_2_'+pl]]=transform_truncated_normal(cube[cube_indeces['t0_2_'+pl]],
                                                                        mu=tcen2,sigma=tdur*0.1,
                                                                        a=tcen2-tdur*0.5,b=tcen2+tdur*0.5)
                cube_indeces['duo_per_int_'+pl]=nparams;nparams+=1
                rel_per_prob = duo['period_aliases']**(-8/3)
                rel_per_prob /= np.sum(rel_per_prob)
                rel_per_prob = np.hstack((0.0,np.cumsum(rel_per_prob)))
                incube=cube[cube_indeces['duo_per_int_'+pl]]
                outcube=incube[:]
                #looping through each gp,p cutting the "cube" up according to their prior probability and making a p~P^-8/3 distribution for each gap:
                for i in range(len(rel_per_prob)-1):
                    ind_min=np.power(self.planets[pl]['per_gaps'][i,1]/self.planets[pl]['per_gaps'][i,0],per_index)
                    outcube[(incube>rel_per_prob[i])&(incube<=rel_per_prob[i+1])]=duo['period_int_aliases'][i]
            if pl in self.multis:
                cube_indeces['per_'+pl]=nparams;nparams+=1
                p=self.planets[pls]['period']
                perr=self.planets[pls]['period_err']
                cube[cube_indeces['per_'+pl]]=transform_normal(cube[cube_indeces['per_'+pl]],
                                                                        mu=p,
                                                                        sigma=np.clip(perr*0.25,0.005,0.02*p))
                #In the case of multitransiting plaets, we know the periods already, so we set a tight normal distribution
            
            ######################################
            #     Initialising R_p & b
            ######################################
            # The Espinoza (2018) parameterization for the joint radius ratio and
            # impact parameter distribution
            rpl=self.planets[pl]['r_pl']/(109.1*self.Rstar[0])
            maxr=1.0 if useL2 else 0.2
            cube_indeces['r_pl_'+pl]=nparams;nparams+=1
            cube[cube_indeces['r_pl_'+pl]] = transform_uniform(cube[cube_indeces['r_pl_'+pl]],0.0,maxr)
            
            cube_indeces['b_'+pl]=nparams;nparams+=1
            cube[cube_indeces['b_'+pl]] = transform_uniform(cube[cube_indeces['b_'+pl]],0.0,1.0)
            #We can do the adjustment for b later when sampling in the model

        ######################################
        #     Initialising Limb Darkening
        ######################################
        # Here we either constrain the LD params given the stellar info, OR we let exoplanet fit them
        if len(np.unique([c[0] for c in self.cads]))==1:
            if constrain_LD:
                n_samples=1200
                # Bounded normal distributions (bounded between 0.0 and 1.0) to constrict shape given star.

                #Single mission
                if 't' in np.unique([c[0].lower() for c in self.cads]):
                    ld_dists=self.getLDs(n_samples=3000,mission='tess')
                    cube_indeces['u_star_tess_0']=nparams;nparams+=1
                    cube[cube_indeces['u_star_tess_0']] = transform_truncated_normal(cube[cube_indeces['u_star_tess_0']],
                                                                                    mu=np.clip(np.nanmedian(ld_dists[:,0],axis=0),0,1),
                                                                                    sd=np.clip(ld_mult*np.nanstd(ld_dists[:,0],axis=0),0.05,1.0),
                                                                                    a=0.0,b=1.0)
                    cube_indeces['u_star_tess_1']=nparams;nparams+=1
                    cube[cube_indeces['u_star_tess_1']] = transform_truncated_normal(cube[cube_indeces['u_star_tess_1']],
                                                                                    mu=np.clip(np.nanmedian(ld_dists[:,0],axis=0),0,1),
                                                                                    sd=np.clip(ld_mult*np.nanstd(ld_dists[:,0],axis=0),0.05,1.0),
                                                                                    a=0.0,b=1.0)
                if 'k' in np.unique([c[0].lower() for c in self.cads]):
                    ld_dists=self.getLDs(n_samples=3000,mission='kepler')
                    cube_indeces['u_star_kep_0']=nparams;nparams+=1
                    cube[cube_indeces['u_star_kep_0']] = transform_truncated_normal(cube[cube_indeces['u_star_kep_0']],
                                                                                    mu=np.clip(np.nanmedian(ld_dists[:,0],axis=0),0,1),
                                                                                    sd=np.clip(ld_mult*np.nanstd(ld_dists[:,0],axis=0),0.05,1.0),
                                                                                    a=0.0,b=1.0)
                    cube_indeces['u_star_kep_1']=nparams;nparams+=1
                    cube[cube_indeces['u_star_kep_1']] = transform_truncated_normal(cube[cube_indeces['u_star_kep_1']],
                                                                                    mu=np.clip(np.nanmedian(ld_dists[:,0],axis=0),0,1),
                                                                                    sd=np.clip(ld_mult*np.nanstd(ld_dists[:,0],axis=0),0.05,1.0),
                                                                                    a=0.0,b=1.0)
            else:
                if 't' in np.unique([c[0].lower() for c in self.cads]):
                    cube_indeces['u_star_tess_0']=nparams;nparams+=1
                    cube[cube_indeces['u_star_tess_0']] = transform_uniform(cube[cube_indeces['u_star_tess_0']],0.0,1.0)
                    cube_indeces['u_star_tess_1']=nparams;nparams+=1
                    cube[cube_indeces['u_star_tess_1']] = transform_uniform(cube[cube_indeces['u_star_tess_1']],0.0,1.0)
                if 'k' in np.unique([c[0].lower() for c in self.cads]):
                    cube_indeces['u_star_kep_0']=nparams;nparams+=1
                    cube[cube_indeces['u_star_kep_0']] = transform_uniform(cube[cube_indeces['u_star_kep_0']],0.0,1.0)
                    cube_indeces['u_star_kep_1']=nparams;nparams+=1
                    cube[cube_indeces['u_star_kep_1']] = transform_uniform(cube[cube_indeces['u_star_kep_1']],0.0,1.0)

        else:
            if constrain_LD:
                n_samples=1200
                #Multiple missions - need multiple limb darkening params:
                ld_dist_tess=self.getLDs(n_samples=3000,mission='tess')

                u_star_tess = pm.Bound(pm.Normal, 
                                       lower=0.0, upper=1.0)("u_star_tess", 
                                                             mu=np.clip(np.nanmedian(ld_dist_tess,axis=0),0,1),
                                                             sd=np.clip(ld_mult*np.nanstd(ld_dist_tess,axis=0),0.05,1.0), 
                                                             shape=2,
                                                             testval=np.clip(np.nanmedian(ld_dist_tess,axis=0),0,1))
                ld_dist_kep=self.getLDs(n_samples=3000,mission='tess')

                u_star_kep = pm.Bound(pm.Normal, 
                                       lower=0.0, upper=1.0)("u_star_kep", 
                                                             mu=np.clip(np.nanmedian(ld_dist_kep,axis=0),0,1),
                                                             sd=np.clip(ld_mult*np.nanstd(ld_dist_kep,axis=0),0.05,1.0), 
                                                             shape=2,
                                                             testval=np.clip(np.nanmedian(ld_dist_kep,axis=0),0,1))
            else:
                # The Kipping (2013) parameterization for quadratic limb darkening paramters
                u_star_tess = xo.distributions.QuadLimbDark("u_star_tess", testval=np.array([0.3, 0.2]))
                u_star_kep = xo.distributions.QuadLimbDark("u_star_kep", testval=np.array([0.3, 0.2]))

        ######################################
        #     Initialising GP kernel
        ######################################
        log_flux_std=np.array([np.log(np.std(self.lc['flux'][self.lc['oot_mask']&(self.lc['cadence']==c)])) for c in self.cads]).ravel()
        logs2 = pm.Normal("logs2", mu = 2*log_flux_std, sd = np.tile(2.0,len(log_flux_std)), shape=len(log_flux_std))

        if use_GP:
            # Transit jitter & GP parameters
            #logs2 = pm.Normal("logs2", mu=np.log(np.var(y[m])), sd=10)
            lcrange=self.lc['time'][self.lc['oot_mask']][-1]-self.lc['time'][self.lc['oot_mask']][0]
            min_cad = np.min([np.nanmedian(np.diff(self.lc['time'][self.lc['oot_mask']&(self.lc['cadence']==c)])) for c in self.cads])
            #freqs bounded from 2pi/minimum_cadence to to 2pi/(4x lc length)
            logw0 = pm.Uniform("logw0",lower=np.log((2*np.pi)/(4*lcrange)), 
                               upper=np.log((2*np.pi)/min_cad),testval=np.log((2*np.pi)/(lcrange)))

            # S_0 directly because this removes some of the degeneracies between
            # S_0 and omega_0 prior=(-0.25*lclen)*exp(logS0)
            maxpower=np.log(np.nanmedian(abs(np.diff(self.lc['flux'][self.lc['oot_mask']]))))+1
            logpower = pm.Uniform("logpower",lower=-20,upper=maxpower,testval=maxpower-6)
            print("input to GP power:",maxpower-1)
            logS0 = pm.Deterministic("logS0", logpower - 4 * logw0)

            # GP model for the light curve
            kernel = xo.gp.terms.SHOTerm(log_S0=logS0, log_w0=logw0, Q=1/np.sqrt(2))

        if not assume_circ:
            # This is the eccentricity prior from Kipping (2013) / https://arxiv.org/abs/1306.4982
            BoundedBeta = pm.Bound(pm.Beta, lower=1e-5, upper=1-1e-5)
            ecc = BoundedBeta("ecc", alpha=0.867, beta=3.03, shape=n_pl,
                              testval=np.tile(0.05,n_pl))
            omega = xo.distributions.Angle("omega", shape=n_pl, testval=np.tile(0.5,n_pl))

        if use_GP:
            self.gp = xo.gp.GP(kernel, self.lc['time'][self.lc['oot_mask']].astype(np.float32),
                               self.lc['flux_err'][self.lc['oot_mask']]**2 + \
                               tt.dot(self.lc['flux_err_index'][self.lc['oot_mask']],tt.exp(logs2)),
                               J=2)

        ################################################
        #     Creating function to generate transits
        ################################################
        def gen_lc(i_orbit,i_r,n_pl,mask=None,prefix=''):
            # Short method to create stacked lightcurves, given some input time array and some input cadences:
            # This function is needed because we may have 
            #   -  1) multiple cadences and 
            #   -  2) multiple telescopes (and therefore limb darkening coefficients)
            trans_pred=[]
            mask = ~np.isnan(self.lc['time']) if mask is None else mask
            if np.sum(self.lc['tele_index'][:,0])>0:
                trans_pred+=[xo.LimbDarkLightCurve(u_star_tess).get_light_curve(
                                                         orbit=i_orbit, r=i_r,
                                                         t=self.lc['time'][mask],
                                                         texp=np.nanmedian(np.diff(self.lc['time'][mask]))
                                                         )/(self.lc['flux_unit']*mult)]
            else:
                trans_pred+=[tt.zeros(( len(self.lc['time'][mask]),n_pl ))]

            if np.sum(self.lc['tele_index'][:,1])>0:
                trans_pred+=[xo.LimbDarkLightCurve(u_star_kep).get_light_curve(
                                                         orbit=i_orbit, r=i_r,
                                                         t=self.lc['time'][mask],
                                                         texp=30/1440
                                                         )/(self.lc['flux_unit']*mult)]
            else:
                trans_pred+=[tt.zeros(( len(self.lc['time'][mask]),n_pl ))]
            # transit arrays (ntime x n_pls x 2) * telescope index (ntime x n_pls x 2), summed over dimension 2
            return pm.Deterministic(prefix+"light_curves", 
                                    tt.sum(tt.stack(trans_pred,axis=-1) * \
                                           self.lc['tele_index'][self.lc['oot_mask']][:,np.newaxis,:],
                                           axis=-1))

        
        

    def myloglikelihood(cube, ndim, nparams):
        x_data, y_data, error_data = args
        mod_data = some_model(cube)
        return np.sum((-0.5 * ((y_data - mod_data) / error_data) ** 2))

    pymultinest.run(myloglike, myprior, n_params)



# model for 2 gaussians, same width, fixed offset
def residuals(params):
    #Adapted from the pymultinest "model" class - produces residual, thereby allowing us to use celerite GPs here
    #input:
    # - Dictionary of parameters with specific names
    
    #Computes transit model with exoplanet
    
    #
    
def prior(cube, ndim, nparams):
    
    
    
    #cube[0] = cube[0]            # uniform prior between 0:1
    cube[1] = 10**(cube[1]*8 - 4) # log-uniform prior between 10^-4 and 10^4
    cube[2] = 10**(cube[2]*4 - 4) # log-uniform prior between 10^-4 and 1
    if ndim < 4:
        return
    cube[3] = 10**(cube[3]*4 - 4) # log-uniform prior between 10^-4 and 1


def loglike(cube, ndim, nparams):
    pos1, width, height1 = cube[0], cube[1], cube[2]
    height2 = cube[3] if ndim > 3 else 0
    ymodel = model(pos1, width, height1, height2)
    loglikelihood = (-0.5 * ((ymodel - ydata) / noise)**2).sum()
    return loglikelihood

parameters = ["pos1", "width", "height1"]
n_params = len(parameters)

pymultinest.run(loglike, prior, n_params, outputfiles_basename='test_1_', resume = False, verbose = True)



In [2]:
import numpy as np
import pymc3 as pm

lb = np.array([0,1,0,2])
ub = np.array([8,9,7,9])
with pm.Model() as model:
    BoundNormal = pm.Bound(pm.Normal,lower=lb, upper=ub)
    a = BoundNormal("a", mu=np.array([4,6,2,7]),sd=np.array([1,1,2,3]),shape=4)
    trace = pm.sample()
    
assert np.all(np.logical_and(trace['a'] >lb, trace['a']<ub))

INFO (theano.gof.compilelock): Waiting for existing lock by process '51462' (I am process '50419')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/hosborn/.theano/compiledir_Darwin-19.3.0-x86_64-i386-64bit-i386-3.7.6-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '51462' (I am process '50419')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/hosborn/.theano/compiledir_Darwin-19.3.0-x86_64-i386-64bit-i386-3.7.6-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '51462' (I am process '50419')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/hosborn/.theano/compiledir_Darwin-19.3.0-x86_64-i386-64bit-i386-3.7.6-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '52866' (I am process '50419')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/hosborn/.theano/compiledir_Darwin-19.3.0-x86_64-i386-64bit